# First Cosmos DB for MongoDB API application

In [ ]:
import os
import pymongo
from pprint import pprint
from dotenv import load_dotenv
from models import Product

## Create a database

In creating a database connection to the MongoDB API, it will create a new database if it does not already exist. In this case, ensure the database connection string is located in a `.env` file in the root of the project, you will need to create this file. The `.env` file should contain the following value (replace the value with your own connection string):

DB_CONNECTION_STRING="mongodb__connection_string"

>**Note**: If you are running using the **local emulator**, append the following value to the connection string: `&retrywrites=false&tlsallowinvalidcertificates=true`. Also ensure that you are starting the emulator using the [MongoDB endpoint option](https://learn.microsoft.com/en-us/azure/cosmos-db/how-to-develop-emulator?tabs=windows%2Cpython&pivots=api-mongodb#start-the-emulator).


In [ ]:
load_dotenv()
CONNECTION_STRING = os.environ.get("DB_CONNECTION_STRING")
client = pymongo.MongoClient(CONNECTION_STRING)
# Create database to hold cosmic works data
# MongoDB will create the database if it does not exist
db = client.cosmic_works

## Create a collection

Creating collections behaves similarly to the database creation. If the collection does not exist, it will be created. It's important to note that databases and collections are lazily created. This means that the database and collection will not be created until the first document is inserted.

In [ ]:
collection = db.products

## Create a document

Documents in Cosmos DB API for MongoDB are represented as JSON objects. In this lab, the Pydantic library is used to create a model for the document. This model is then used to create a document in the database using built-in serialization methods. Find the models in the `models` folder. Notice the class property definitions include aliases, these aliases can be used to override the serialized property names. This is useful when the property names in the model do not match the property names desired in the database.

One method of creating a document is using the `insert_one` method. This method takes a single document and inserts it into the database. This operation returns an [InsertOneResult](https://pymongo.readthedocs.io/en/stable/api/pymongo/results.html#pymongo.results.InsertOneResult) object that contains the property `inserted_id`. This property contains the unique identifier of the document that was just inserted.

In [ ]:
product = Product(
        id="2BA4A26C-A8DB-4645-BEB9-F7D42F50262E",    
        category_id="56400CF3-446D-4C3F-B9B2-68286DA3BB99", 
        category_name="Bikes, Mountain Bikes", 
        sku="BK-M18S-42",
        name="Mountain-100 Silver, 42",
        description='The product called "Mountain-500 Silver, 42"',
        price=742.42,
       )

# Generate JSON using alias names defined on the model
product_json = product.model_dump(by_alias=True)

# Insert the JSON into the database, and retrieve the inserted/generated ID
product_id = collection.insert_one(product_json).inserted_id

print(f"Inserted product with ID: {product_id}")

## Read a document

The insertion of the Product in the previous cell automatically created the database and collection. The `find_one` method is used to retrieve a single document from the database. The `find_one` method takes a filter as an argument. This filter is used to find the document in the database. In this case, the filter is the unique identifier or `_id` of the document that was just inserted.

In [ ]:
retrieved_document = collection.find_one({"_id": product_id})

# Print the retrieved JSON document
print("JSON document retrieved from the database:")
pprint(retrieved_document)

# Cast JSON document into the Product model
retrieved_product = Product(**retrieved_document)

# Print the retrieved product
print("\nCast Product from document:")
print(retrieved_product)

## Update a document

The `find_one_and_update` method is used to update a single document in the database. This method takes a filter and an update as arguments. The filter is used to find the document to update. The update is a dictionary of the properties to update. In this case, the `find_one_and_update` method is used to update the `name` property of the document. The updated document is the return value for this method.

Find additional examples of queries in the [documentation](https://learn.microsoft.com/en-us/azure/cosmos-db/mongodb/tutorial-query).

In [ ]:
retrieved_product.name = "Mountain-100 Silver, 48\""
update_result = collection.find_one_and_update(
    {"_id": product_id},
    {"$set": {"name": retrieved_product.name}},
    return_document=pymongo.ReturnDocument.AFTER
)
print("Updated JSON document:")
print(update_result)
updated_product = Product(**update_result)
print(f"\nUpdated Product name: {updated_product.name}")

## Delete a document

The `delete_one` method is used to delete a single document from the database. This method takes a filter as an argument. This filter is used to find the document to delete. In this case, the filter is the unique identifier or `_id` of the document that was just inserted and updated.

In [ ]:
delete_result = collection.delete_one({"_id": product_id})
print(f"Deleted documents count: {delete_result.deleted_count}")
print(f"Number of documents in the collection: {collection.count_documents({})}")

## Query for multiple documents

The `find` method is used to query for multiple documents in the database. This method takes a filter as an argument. This filter is used to find the documents to return. In this case, the filter is an empty dictionary. This will return all documents in the collection.

In [ ]:
# Insert multiple documents
products = [
    Product(
        id="2BA4A26C-A8DB-4645-BEB9-F7D42F50262E",    
        category_id="56400CF3-446D-4C3F-B9B2-68286DA3BB99", 
        category_name="Bikes, Mountain Bikes", 
        sku="BK-M18S-42",
        name="Mountain-100 Silver, 42",
        description='The product called "Mountain-500 Silver, 42"',
        price=742.42
       ),
    Product(
        id="027D0B9A-F9D9-4C96-8213-C8546C4AAE71",    
        category_id="26C74104-40BC-4541-8EF5-9892F7F03D72", 
        category_name="Components, Saddles", 
        sku="SE-R581",
        name="LL Road Seat/Saddle",
        description='The product called "LL Road Seat/Saddle"',
        price=27.12
       ),
    Product(
        id = "4E4B38CB-0D82-43E5-89AF-20270CD28A04",
        category_id = "75BF1ACB-168D-469C-9AA3-1FD26BB4EA4C",
        category_name = "Bikes, Touring Bikes",
        sku = "BK-T44U-60",
        name = "Touring-2000 Blue, 60",
        description = 'The product called Touring-2000 Blue, 60"',
        price = 1214.85
       ),
    Product(
        id = "5B5E90B8-FEA2-4D6C-B728-EC586656FA6D",
        category_id = "75BF1ACB-168D-469C-9AA3-1FD26BB4EA4C",
        category_name = "Bikes, Touring Bikes",
        sku = "BK-T79Y-60",
        name = "Touring-1000 Yellow, 60",
        description = 'The product called Touring-1000 Yellow, 60"',
        price = 2384.07
       ),
    Product(
        id = "7BAA49C9-21B5-4EEF-9F6B-BCD6DA7C2239",
        category_id = "26C74104-40BC-4541-8EF5-9892F7F03D72",
        category_name = "Components, Saddles",
        sku = "SE-R995",
        name = "HL Road Seat/Saddle",
        description = 'The product called "HL Road Seat/Saddle"',
        price = 52.64,
       )
]

# The bulk_write method takes a list of write operations and executes them in the same transaction
# The UpdateOne operation updates a single document, notice the upsert=True option, this means that
# if the document does not exist, it will be created, if it does exist, it will be updated
collection.bulk_write([pymongo.UpdateOne({"_id": prod.id}, {"$set": prod.model_dump(by_alias=True)}, upsert=True) for prod in products])

In [ ]:
# Print all documents that have a category name of "Components, Saddles"
for result in collection.find({"categoryName": "Components, Saddles"}):
    pprint(result)

In [ ]:
# Print all documents that have a category name that includes the word "Bikes"
for result in collection.find({"categoryName": {"$regex": "Bikes"}}):
    pprint(result)

## Clean up resources

The following cell will delete the database and collection created in this lab. This is done by using the `drop_database` method on the database object. This method takes the name of the database to delete as an argument. If it is desired to simply delete the collection, the `drop_collection` method can be used on the database object. This method takes the name of the collection to delete as an argument.

In [ ]:
# db.drop_collection("products")
client.drop_database("cosmic_works")
client.close()